# Fine-tuning ST SSD MobileNet v1 0.25 on FreLon COCO

This notebook prepares the STMicroelectronics SSD MobileNet v1 0.25 baseline for fine-tuning on the FreLon dataset. The original Windows-only TensorFlow Lite loading sequence has been replaced with logic that downloads the `.h5` weights alongside the YAML configuration shipped in the STM32 model zoo. Data loading has also been updated to materialize uniform tensors from the ragged `.npz` annotations so they can be fed into training pipelines more easily.


## Environment setup
Import packages, configure reproducibility, and describe file-system constants used throughout the notebook.

In [ ]:
import ast
import os
from pathlib import Path
from typing import Dict, Iterable, List, Tuple

import numpy as np
import requests
import tensorflow as tf
import tf_keras
import yaml

print(f"TensorFlow version: {tf.__version__}")
print(f"tf_keras version: {tf_keras.__version__}")

In [ ]:
# Ensure deterministic behaviour where possible
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)


In [ ]:
# Paths and remote resources
NOTEBOOK_DIR = Path.cwd()
DATA_ROOT = NOTEBOOK_DIR / "hornet-bees-2"
MODEL_VARIANT = "st_ssd_mobilenet_v1_025_256"
MODEL_BASE_URL = (
    "https://raw.githubusercontent.com/STMicroelectronics/stm32ai-modelzoo/main/"
    "object_detection/st_ssd_mobilenet_v1/ST_pretrainedmodel_public_dataset/coco_2017_person"
)
MODEL_CACHE = NOTEBOOK_DIR / "models_cache" / MODEL_VARIANT
MODEL_CACHE.mkdir(parents=True, exist_ok=True)

MODEL_FILES = {
    "weights": f"{MODEL_VARIANT}.h5",
    "config": f"{MODEL_VARIANT}_config.yaml",
}


def download_file(url: str, destination: Path) -> Path:
    "Download a file from a URL if it does not already exist."
    if destination.exists():
        return destination
    response = requests.get(url, timeout=60)
    response.raise_for_status()
    destination.write_bytes(response.content)
    return destination


def ensure_model_assets() -> Dict[str, Path]:
    "Retrieve model weights and configuration from the STM32 model zoo."
    assets: Dict[str, Path] = {}
    for key, filename in MODEL_FILES.items():
        url = f"{MODEL_BASE_URL}/{MODEL_VARIANT}/{filename}"
        path = MODEL_CACHE / filename
        assets[key] = download_file(url, path)
    return assets


def parse_input_shape(raw_shape: str) -> Tuple[int, int, int]:
    "Parse the (H, W, C) tuple stored as a string in the YAML config."
    shape = ast.literal_eval(raw_shape)
    if len(shape) != 3:
        raise ValueError(f"Unexpected input shape {shape}")
    return tuple(int(dim) for dim in shape)


def ensure_materialized_npz(npz_path: Path) -> None:
    "Validate that Git LFS placeholders have been pulled before loading numpy archives."
    if not npz_path.exists():
        raise FileNotFoundError(f"Expected dataset file missing: {npz_path}")
    head = npz_path.read_bytes()[:64]
    if b"version https://git-lfs.github.com" in head:
        raise RuntimeError(
            f"{npz_path} is a Git LFS pointer. Run `git lfs pull` or download the FreLon dataset "
            "before executing this notebook."
        )


In [ ]:
# Download assets and instantiate the baseline SSD Mobilenet model
assets = ensure_model_assets()
print("Model assets cached:", assets)

with assets["config"].open("r", encoding="utf-8") as cfg_file:
    model_config = yaml.safe_load(cfg_file)

input_shape = parse_input_shape(model_config["training"]["model"]["input_shape"])
class_names = model_config.get("dataset", {}).get("class_names", ["hornet"])
print(f"Input shape: {input_shape}")
print(f"Classes declared in config: {class_names}")

# Load the STMicro `.h5` model using the legacy-compatible `tf_keras` loader
st_model = tf_keras.models.load_model(assets["weights"], compile=False)
print("Model outputs:", st_model.output_names)
print("Detection heads shapes:", [output.shape for output in st_model.outputs])


## Data pipeline helpers
Functions below load the FreLon `.npz` exports, normalise the images, and convert ragged label structures into padded tensors. The padded representation captures bounding boxes, one-hot encoded classes, and a mask marking real boxes versus padding so downstream training code can create dense tensors matching the detection heads.

In [ ]:
def load_split(split: str) -> Tuple[np.ndarray, List[Dict[str, np.ndarray]]]:
    "Load images and raw annotations for a given split."
    npz_path = DATA_ROOT / f"{split}_frelon.npz"
    ensure_materialized_npz(npz_path)
    with np.load(npz_path, allow_pickle=True) as data:
        images = data["X"].astype(np.float32)
        annotations = list(data["y"])
    return images, annotations


def normalise_images(images: np.ndarray) -> np.ndarray:
    return images / 255.0


def pad_annotations(
    annotations: List[Dict[str, np.ndarray]],
    num_classes: int,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    "Convert ragged detection targets into dense arrays."
    max_boxes = max(len(entry["boxes"]) for entry in annotations)
    batch_size = len(annotations)
    boxes = np.zeros((batch_size, max_boxes, 4), dtype=np.float32)
    classes = np.zeros((batch_size, max_boxes, num_classes), dtype=np.float32)
    mask = np.zeros((batch_size, max_boxes), dtype=np.float32)
    for idx, entry in enumerate(annotations):
        entry_boxes = np.asarray(entry["boxes"], dtype=np.float32)
        entry_class_ids = np.asarray(entry.get("class_ids", entry.get("classes")), dtype=np.int32)
        count = len(entry_boxes)
        if count == 0:
            continue
        boxes[idx, :count] = entry_boxes
        classes[idx, np.arange(count), entry_class_ids] = 1.0
        mask[idx, :count] = 1.0
    return boxes, classes, mask


def summarise_split(name: str, images: np.ndarray, annotations: List[Dict[str, np.ndarray]], num_classes: int) -> None:
    boxes, classes, mask = pad_annotations(annotations, num_classes)
    print(f"{name} images: {images.shape}")
    print(f"{name} boxes tensor shape: {boxes.shape}")
    print(f"{name} classes tensor shape: {classes.shape}")
    print(f"{name} mask tensor shape: {mask.shape}")


In [ ]:
# Attempt to materialise the FreLon dataset splits
try:
    train_images, train_ann = load_split("train")
    val_images, val_ann = load_split("val")
    test_images, test_ann = load_split("test")
    summarise_split("Train", train_images, train_ann, len(class_names))
    summarise_split("Validation", val_images, val_ann, len(class_names))
    summarise_split("Test", test_images, test_ann, len(class_names))
except Exception as error:
    print(f"Dataset unavailable: {error}")
    train_images = val_images = test_images = None
    train_ann = val_ann = test_ann = None


## Fine-tuning workflow (placeholder)
The FreLon data can now be converted into dense tensors, but training requires anchor matching against the SSD detection heads. Implementing that matching logic is outside the scope of this automated refactor. The cell below illustrates where a fine-tuning loop would live once the dataset has been fully materialised and encoded for the SSD outputs.

In [ ]:
if train_images is None:
    raise RuntimeError(
        "Training cannot start because the FreLon dataset archives were not available. "
        "Download the `.npz` files (or run `git lfs pull`) and re-run the data-loading cell."
    )

# TODO: Implement SSD anchor matching and compile the model with appropriate losses.
# Placeholder to indicate where fine-tuning would be triggered once targets are aligned with the 6,825 anchors.
# st_model.compile(...)
# history = st_model.fit(...)
